In [1]:
#!pip install -r requirements.txt -q

In [2]:
import gmpy2
import numpy as np
from functools import partial
from sage.libs.pari.all import pari
from sage.all import ZZ, PolynomialRing, GF

from preprocessing.functions import ParamGen, KeyGen
from preprocessing.encryption import decrypt, encrypt
from preprocessing.message import decode, encode, prepare_slots
from protocols.zpk_o_pk import zk_pok_prover_fiat_shamir, zk_pok_verify_fiat_shamir

pari.allocatemem(2 * 1024**3)

PARI stack size set to 2147483648 bytes, maximum size set to 2147483648


# Generación de parámetros y claves para el esquema homomórfico / ZKP

Esta celda realiza tres pasos principales:

1. **Fijar semilla y generador aleatorio**  
   - `SEED` controla la reproducibilidad.  
   - `rng = np.random.default_rng(SEED)` es el generador de NumPy que usamos para todas las decisiones aleatorias reproducibles.

2. **Definición de parámetros básicos**  
   - `n`: (tamaño de polinomios / dimensión, etc.). En el ejemplo `n = 3` (se usa solo como ejemplo).
   - `p`: primo base que define el cuerpo `F_p` (aquí se construye con `gmpy2.mpz(2)**16 + 1` como ejemplo; sustituir por el primo real que necesites).
   - `k, s, sec`: parámetros adicionales del sistema:
     - `k` (grado de extensión o similar),  
     - `s` (número de copias, réplicas o slots requeridos),  
     - `sec` (parámetro relacionado con la seguridad / número de repeticiones/desafíos; en tu código usas `sec=40`).

3. **Generación de parámetros y claves**  
   - `ParamGen(...)` — función que calcula parámetros internos del esquema (Aq, tau, delta, m, N, coefs, C_m, q, r).  
     - `Aq`: el anillo cociente resultante (por ejemplo, `Z_q[x]/(f(x))`).  
     - `m`, `phi(m)`, `coeffs` y `C_m`: información sobre el polinomio m y su factorización/coeficientes.  
     - `q`: módulo final recomendado (gran entero).  
     - `r`: ruido / desviación típica usada en el esquema.
   - `KeyGen(...)` — función que genera `pk` (public key), `sk` (secret key) y `pk_hat` (posible clave auxiliar).

## Lectura de salida (qué significan las líneas impresas)
- `m = 32768 phi(m) = 16384 degrees: [...]`  
  Indica la estructura del módulo/polinomio que define el anillo (grado y coeficientes).
- `m es primo, asi que C_m = ...`  
  Mensaje informativo sobre la primalidad y alguna constante `C_m`.
- `Convergencia en iteración 1`  
  Indica que el algoritmo de optimización de parámetros convergió rápido.
- **RESULTADOS DE PARÁMETROS OPTIMIZADOS**  
  - `Módulo (q)` : el valor de `q` (módulo) y su entropía aproximada en potencias de 2.  
  - `Ruido std (r/rho)` : la desviación estándar del ruido — debe cumplir un umbral mínimo.
- `Construido A_q = Univariate Quotient Polynomial Ring ...`  
  Se confirmó la construcción del anillo sobre `Z_q`.

In [3]:
SEED = 777
rng = np.random.default_rng(SEED)

n = 3
# 2**64 + 13
p = gmpy2.mpz(2)**16 + 1  # primo que define el cuerpo de base 𝔽_p
k = 1   # queremos incluir 𝔽_{p^k} dentro de A
s = 3   # número de copias de 𝔽_{p^k} que necesitamos
sec = 40

Aq, tau, delta, m, N, coeffs, C_m, q, r = ParamGen(p, k, s, n, sec)

pk, sk, pk_hat = KeyGen(Aq, N, r, q, rng, p)

m = 32768 phi(m) = 16384 degrees: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

# Prueba de Encode / Decode

Esta celda demuestra cómo tomar una lista de valores (mensajes), codificarlos en un **elemento del anillo** `A_q` mediante la función `encode(...)` y luego recuperarlos con `decode(...)` para comprobar que la operación es inversa.

**Flujo principal:**
1. `prepare_slots(p, coeffs)` — prepara la estructura `R_p` y la lista `slots_moduli` que definen cómo se empaquetan los mensajes en `A_q` (cantidad de *slots* y sus módulos).
2. Definimos el `plaintext` como una **lista de enteros**, aquí usando `ord()` sobre caracteres para convertir texto en enteros.
3. `encode(plaintext, R_p, slots_moduli, Aq)` — empaqueta los enteros en un elemento del anillo `Aq`.
4. `decode(plaintext_poly, R_p, slots_moduli)` — recupera la lista de enteros desde el elemento codificado.
5. Comprobamos que los primeros `k` elementos recuperados coincidan con el original (en el ejemplo usamos los primeros 4).

In [4]:
print("\n--- Iniciando Prueba de Encoding/Decoding ---")

# 1. Preparar la estructura SIMD
R_p, slots_moduli = prepare_slots(p, coeffs)
num_slots = len(slots_moduli)
print(f"Sistema configurado con {num_slots} slots disponibles.")

# 2. Definir mensajes de prueba
# Solo usaremos 4 valores, el resto serán ceros
plaintext = [ord("H"), ord("o"), ord("l"), ord("a")] 

print(f"Mensajes originales: {plaintext}")

# 3. Codificar (Encode)
plaintext_poly = encode(plaintext, R_p, slots_moduli, Aq)

print("Codificación exitosa. Elemento en Aq creado.")
# print(plaintext_poly) # Descomentar para ver el polinomio gigante

# 4. Decodificar (Decode) para verificar
decoded_message = decode(plaintext_poly, R_p, slots_moduli)

# Filtramos solo los primeros 4 para comparar visualmente
print(f"Mensajes recuperados: {decoded_message[:4]}")

# Verificación automática
assert decoded_message[:4] == plaintext
print("¡Éxito! Los mensajes coinciden.")


--- Iniciando Prueba de Encoding/Decoding ---
Sistema configurado con 16384 slots disponibles.
Mensajes originales: [72, 111, 108, 97]
Codificación exitosa. Elemento en Aq creado.
Mensajes recuperados: [72, 111, 108, 97]
¡Éxito! Los mensajes coinciden.


## Encrypt/Decrypt

In [5]:
# Encryption
ciphertext = encrypt(plaintext, N, r, q, rng, R_p, slots_moduli, Aq, p, pk)

# Decryption
decrypted_messages = decrypt(ciphertext, R_p, slots_moduli, sk)

# Verificación automática
assert decrypted_messages[:4] == plaintext
print("¡Éxito! Los mensajes coinciden.")

¡Éxito! Los mensajes coinciden.


## ZKPoPK

In [6]:
# Generate parameters for ZKP
chars = list(range(65, 91)) + list(range(97, 123))  # Aa–Zz
messages = rng.choice(chars, size=sec)

witness_xr = []
ciphertexts_c = []

for i, msg in enumerate(messages):
    # avisar cada 5 mensajes (empezando en el 5)
    if (i + 1) % 10 == 0:
        print(f"[INFO] Procesados {i+1} mensajes de {sec}")

    ci, xi, ri = encrypt(plaintext, N, r, q, rng, R_p, slots_moduli, Aq, p, pk, verbose=True)
    witness_xr.append((xi, ri))
    ciphertexts_c.append(ci)


[INFO] Procesados 10 mensajes de 40
[INFO] Procesados 20 mensajes de 40
[INFO] Procesados 30 mensajes de 40
[INFO] Procesados 40 mensajes de 40


In [7]:
# Ejecutar ZKP
transcript = zk_pok_prover_fiat_shamir(pk,
                                        ciphertexts_c,
                                        witness_xr,
                                        partial(rng.choice, chars, size=sec),
                                        sec,
                                        N, r, q, rng, R_p, slots_moduli, Aq, p,
                                        serialize_ciphertext=lambda C: str(C).encode())

print("transcript terminado")

ok = zk_pok_verify_fiat_shamir(pk, ciphertexts_c, transcript,
                               N, r, q, rng, R_p, slots_moduli, Aq, p)
print("Verificación:", ok)

transcript terminado
Verificación: True


# 🧪 Test de Integración: Protocolo de Preprocesamiento (Offline Phase)

Esta celda ejecuta una **prueba de integración completa** para la fase de preprocesamiento del protocolo MPC (basado en el esquema DPSZ/BGV). El objetivo es validar que los sub-protocolos (`Initialize`, `Pair`, `Triple`) interactúan correctamente y generan material criptográfico válido.

### ⚙️ Configuración del Entorno de Prueba
Se utilizan parámetros "dummy" reducidos para una ejecución rápida y verificable:
* **Anillo y Cuerpo:** $p=17$, $k=1$ (Cuerpo primo $\mathbb{F}_{17}$ para facilitar la verificación aritmética).
* **Parámetros BGV:** $N=8$ (Grado ciclotómico muy bajo), $q=2^{40}$ (Módulo grande para evitar desbordamiento de ruido durante el test).
* **Simulación:** Se simulan las claves (`pk`, `sk`) y el entorno de red de $n=3$ jugadores.

### 🔍 Flujo de la Prueba
El script ejecuta secuencialmente las siguientes fases:

1.  **Inicialización (`run_initialize`):**
    * Genera la clave global MAC $\alpha$ y las claves personales $\beta_i$.
    * **Verificación:** Desencripta el cifrado de $\alpha$ usando la clave secreta `sk` (solo para propósitos de test) para confirmar que es recuperable.

2.  **Generación de Pares (`run_pair`):**
    * Genera tuplas $(r, [r], \langle r \rangle)$ para el enmascaramiento de entradas.
    * **Verificación:** Confirma que el protocolo finaliza y devuelve los shares de $r$.

3.  **Generación de Triples de Beaver (`run_triple`):**
    * Genera tuplas $(a, b, c)$ tales que $c = a \cdot b$.
    * **Verificación (Crítica):**
        * Reconstruye los secretos $a, b, c$ sumando los shares de los $n$ jugadores módulo $p$.
        * Comprueba matemáticamente si se cumple la propiedad multiplicativa:
            $$a \cdot b \equiv c \pmod p$$

### 📊 Interpretación de Resultados
* ✅ **ÉXITO:** Verás el mensaje `¡ÉXITO! El triple es válido: a * b = c`. Esto confirma que la homomorfía, el protocolo `reshare` y la gestión de ruido funcionan correctamente.
* ❌ **FALLO:** Si obtienes `ERROR: ... != ...`, indica que el ruido ha corrompido el cifrado (prueba aumentar $q$) o que hay un error lógico en la suma/resta de los shares en `reshare`.

---

In [9]:
from protocols.prep import PreprocessingProtocol


context = {
        'N': N, 'r': r, 'q': q, 'p': p, 'k': k, 'n': n, 's': s,
        'rng': np.random.default_rng(),
        'Aq': Aq, 'R_p': R_p, 'slots_moduli': slots_moduli, # Rellenar si tu decode lo usa
        'pk': pk, 'sk': sk
    }
    
# Instanciar protocolo
print("\n[1] Instanciando PreprocessingProtocol...")
proto = PreprocessingProtocol(context)

# ------------------------------------------------------
# 2. Test Initialize (Generación de Claves Alpha y Beta)
# ------------------------------------------------------
print("\n[2] Ejecutando 'run_initialize'...")
# bracket_alpha es una matriz donde fila i son los shares de gamma_i
bracket_alpha = proto.run_initialize()
    
# VERIFICACIÓN:
# [alpha] significa que tenemos shares de gamma_i tales que sum(gamma_i) = alpha * beta_i
print("    > Verificando estructura [alpha]...")
    
# a) Recuperar alpha y betas desencriptando (TRUCO DE TEST)
# proto.e_alpha es el cifrado de la suma de alphas
alpha_poly = decrypt(proto.e_alpha, context['R_p'], context["slots_moduli"], sk) 
# Nota: decrypt devuelve polinomio, asumimos que representa el valor en los slots
# Para este test simplificado, asumiremos que encode/decode funciona o miramos coeficientes
    
# b) Chequear consistencia de Bracket
# Para cada jugador 'i' (dueño de beta_i), la suma de los shares gamma que tienen TODOS
# debe ser igual a alpha * beta_i.
    
# Como beta_i está cifrado en proto.e_beta_list[i], lo desciframos para chequear
beta_i_polys = [decrypt(ct, context['R_p'], context["slots_moduli"], sk) for ct in proto.e_beta_list]
    
# Chequeo algebraico (aprox en slots o exacto si es escalar repetido)
# Si la encriptación funcionó, alpha_poly * beta_i_poly ≈ gamma_reconstruida
print("    ✅ Initialize finalizado (si no hubo error de ejecución).")

# ------------------------------------------------------
# 3. Test Pair (Generación de r)
# ------------------------------------------------------
print("\n[3] Ejecutando 'run_pair'...")
# Asegúrate de haber modificado run_pair para devolver r_i_list
r_i_list, bracket_r, angle_r = proto.run_pair()
    
# Reconstruir r (suma de r_i)
# r_total = sum(r_i_list) (vectorial)
print(f"    > Pair generado. Shares de r recuperados para {len(r_i_list)} jugadores.")
    
# Verificar <r>: sum(shares_gamma) == r * alpha
# (Esto requeriría operar con los polinomios descifrados o los valores Fpk)
    
# ------------------------------------------------------
# 4. Test Triple (Generación de a, b, c) - EL MÁS IMPORTANTE
# ------------------------------------------------------
print("\n[4] Ejecutando 'run_triple'...")
a_i, b_i, c_i, ang_a, ang_b, ang_c = proto.run_triple()
    
print("    > Verificando relación multiplicativa c = a * b ...")
    
# Reconstruir valores planos (sumando los vectores de los jugadores)
def reconstruct(shares_list):
    # Suma vectorial simple modulo p (asumiendo Fp para simplificar test)
    total = np.zeros(s, dtype=int)
    for share in shares_list:
        # share es lista de s elementos
        total = (total + np.array(share)) % p
    return total

# Reconstruir a, b, y c desde los shares que devolviste
a_val = reconstruct(a_i)
b_val = reconstruct(b_i)
c_val = reconstruct(c_i)
    
print(f"      a reconstruido: {a_val}")
print(f"      b reconstruido: {b_val}")
print(f"      c reconstruido: {c_val}")
    
# Chequeo: c == a * b mod p (o en F_pk)
expected_c = (a_val * b_val) % p
    
if np.array_equal(c_val, expected_c):
    print("    ✅ ¡ÉXITO! El triple cumple a * b = c")
else:
    print(f"    ❌ ERROR: {a_val} * {b_val} != {c_val}")
    print("       (Nota: Si falla, revisa el ruido 'r' o la función 'reshare')")


[1] Instanciando PreprocessingProtocol...

[2] Ejecutando 'run_initialize'...
--- Running Initialize ---
Initialize completado. Claves generadas.
    > Verificando estructura [alpha]...
    ✅ Initialize finalizado (si no hubo error de ejecución).

[3] Ejecutando 'run_pair'...
--- Running Pair ---
    > Pair generado. Shares de r recuperados para 3 jugadores.

[4] Ejecutando 'run_triple'...
--- Running Triple ---
    > Verificando relación multiplicativa c = a * b ...
      a reconstruido: [8321 58578 60469]
      b reconstruido: [11584 15604 12123]
      c reconstruido: [51074 6573 34342]
    ✅ ¡ÉXITO! El triple cumple a * b = c
